In [2]:
using BenchmarkTools: @btime
import LinearAlgebra as la
using TensorOperations: @tensor

path = "../../"
include(path * "Systems/system.jl")
include(path * "Methods/HF.jl")
include(path * "Methods/CCD.jl")
include(path * "Methods/CCSD.jl")

LoadError: SystemError: opening file "C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods\\Systems\\system.jl": No such file or directory

In [3]:
using PyCall

In [4]:
py"""
import numpy as np
import scipy.special as spec
"""
@pyinclude("two_body_integrals.py")
@pyinclude("transform.py")
@pyinclude("HF.py")

### Setup

In [76]:
l = 5
ω = 0.25
basis = SpinBasis(HOBasis(l, ω))

n = 2
V = ShieldedCoulomb(0.25)
grid = [x for x in range(-10, stop = 10, length = 201)]
spfs = spatial(basis, grid);

In [1]:
spfs

LoadError: UndefVarError: spfs not defined

In [78]:
h = onebody(basis, grid);

### Inner ints

In [27]:
my_inner = inner_ints(spfs, grid, V::Interaction);

In [28]:
spf = reduce(vcat,transpose.(spfs));
py_inner = py"_compute_inner_integral"(spf, basis.l, length(grid), Vector(grid), 1, 0.25);

In [29]:
py_inner ≈ my_inner

true

### Outer ints

In [31]:
my_u = outer_int(spfs, grid, my_inner);

In [32]:
py_u = py"_compute_orbital_integrals"(spf, basis.l, py_inner, Vector(grid));

In [33]:
py_u ≈ my_u

true

### Add spin

In [80]:
my_us = add_spin_u(my_u);

In [84]:
py_us = py"add_spin_two_body"(py_u);

In [88]:
my_us ≈ py_us

true

### Transform

In [37]:
C = rand(basis.l, basis.l);

In [39]:
my_tu = transform_twobody(u, C);

In [40]:
py_tu = py"transform_two_body_elements"(py_u, C);

In [42]:
my_tu ≈ py_tu

true

### Trapz

In [43]:
f = rand(100);
x = rand(100);

In [49]:
my_trapz = trapz(f, x);

In [50]:
py_trapz = py"_trapz"(f, x);

In [51]:
my_trapz ≈ py_trapz

true

### Hartree-Fock

In [59]:
function getP(C, n, l)
    P = zero(C)
    for a in 1:l
        for b in 1:l
            @inbounds P[b, a] = 0
        end
    end
    
    for i in 1:n
        for a in 1:l
            for b in 1:l
                @inbounds P[b, a] += conj(C[a, i]) * C[b, i]
            end
        end
    end
    return P
end

function getF(P, n, l, h, u)
    F = zero(P)
    F .= h
    for c in 1:l
        for d in 1:l
            @inbounds P_dc = P[d, c]
            for a in 1:l
                for b in 1:l
                    @inbounds F[a, b] += P_dc * u[a, c, b, d]
                end
            end
        end
    end
    return F
end

getF (generic function with 1 method)

In [199]:
C = Matrix{Float64}(la.I(basis.l));

### P

In [200]:
my_P = getP(C, n, basis.l);

In [201]:
py_P = py"getP"(C, n);

In [202]:
my_P ≈ py_P

true

### F

In [203]:
my_F = getF(my_P, n, basis.l, h, my_u);

In [204]:
py_F = py"getF"(py_P, h, py_u, n);

In [205]:
my_F ≈ py_F

true

### HF-Update

In [206]:
my_C2 = la.eigvecs(my_F);

In [207]:
py_C2 = py"np.linalg.eigh"(py_F)[2];

In [224]:
abs.(my_C2) ≈ abs.(py_C2) # The signs of the eigenvectors might not align

true

In [225]:
my_P2 = getP(my_C2, n, basis.l);
my_F2 = getF(my_P2, n, basis.l, h, my_u);
my_C3 = la.eigvecs(my_F2);

In [226]:
py_P2 = py"getP"(py_C2, n);
py_F2 = py"getF"(py_P2, h, py_u, n);
py_C3 = py"np.linalg.eigh"(py_F2)[2];

In [227]:
my_P2 ≈ py_P2

true

In [228]:
my_F2 ≈ py_F2

true

In [229]:
abs.(my_C3) ≈ abs.(py_C3)

false

### Spfs

In [128]:
py"""
def ho_function(x, n, omega):
    return (
        normalization(n, omega)
        * np.exp(-0.5 * omega * x ** 2)
        * spec.hermite(n)(np.sqrt(omega) * x)
    )

def normalization(n, omega):
    return (
        1.0
        / np.sqrt(2 ** n * spec.factorial(n))
        * (omega / np.pi) ** 0.25
    )
"""

In [110]:
x = Vector(grid);

In [139]:
py"ho_function"(x, 0, ω) ≈ spfs[1]

true

In [148]:
py"ho_function"(x, 1, ω) ≈ spfs[3]

true